In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
from bs4 import BeautifulSoup

In [3]:
# min_id = 175660
# max_id = 256119
# ids = np.arange(min_id, max_id + 1).astype(str)

In [2]:
min_id = 269999
max_id = 323767
ids = np.arange(min_id, max_id + 1).astype(str)

In [3]:
df = pd.DataFrame({'url': np.array(list(map(lambda x: "https://www.5-tv.ru/news/" + x, ids)))})
df.index = np.arange(min_id, max_id + 1)
df.to_json('news_5tv.json')
df

,url
269999,https://www.5-tv.ru/news/269999
270000,https://www.5-tv.ru/news/270000
270001,https://www.5-tv.ru/news/270001
270002,https://www.5-tv.ru/news/270002
270003,https://www.5-tv.ru/news/270003
...,...
323763,https://www.5-tv.ru/news/323763
323764,https://www.5-tv.ru/news/323764
323765,https://www.5-tv.ru/news/323765
323766,https://www.5-tv.ru/news/323766


# Parsing

In [5]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    header = soup.find('p', attrs={'class': 'fsHeader'})
    news_text = header.get_text(" ", strip=True) if header else ''
    
    news_body = soup.find(name='article', attrs={'class': 'js-mediator-article'})
    if news_body:
        bad = news_body.find_all(name=['script', 'iframe', 'svg', 'figure'])
        if bad:
            [tag.decompose() for tag in bad]
        
        news_ps = news_body.find_all(name=['p', 'li'])
        if news_ps:
            news_text += " " + " ".join([news_p.get_text(" ", strip=True) for news_p in news_ps])

    news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')

    lastmod_elem = soup.find(name="meta", attrs={'name': "mediator_published_time"})
    lastmod = re.sub(r'[\n\r]+', ' ', lastmod_elem.get("content")) if lastmod_elem else ''
    if lastmod:
        lastmod = pd.to_datetime(lastmod, utc=True).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    tags_elem = soup.find(name="meta", attrs={'name': "news_keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get("content")) if tags_elem else ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+|<[^>]+>', ' ', description_elem.get('content')).replace(u'\xa0', u'') if description_elem else ''
        
    return {'lastmod': lastmod, 'news_text': news_text, 'tags': tags, 'description': description}

In [6]:
pd.to_datetime('2018-07-27T08:17:00+0300', utc=True).strftime("%Y-%m-%dT%H:%M:%SZ")

'2018-07-27T05:17:00Z'

In [7]:
parse_news(req.get('https://www.5-tv.ru/news/213787').content)

{'lastmod': '2018-07-27T05:17:00Z',
 'news_text': ' Международный кубок чемпионов в самом разгаре. В Сингапуре встретились испанцы и англичане. Отыграв основное время вничью, команды выяснили, кто из них сильнее, с помощью пенальти. Пятый канал приводил лучшие моменты матча . Лунное затмение 27 июля 2018 года станет самым длинным в XXI веке. Но вместе с этим россияне смогут увидеть еще одно интереснейшее астрономическое явление — Марс очень сильно приблизится к Земле. Следующего такого случая никто из нас уже не дождется, поскольку это произойдет через 25 миллионов лет. Это явно стоит того, чтобы вооружиться сегодня телескопом или хотя бы биноклем.  Страшная авария произошла в Подмосковье. Огромная фура протаранила сразу шесть автомобилей. ВАЗ-2108 мгновенно загорелся — внутри находились водитель и трое пассажиров. Спасти их не удалось. Но только после того, как ее отец полностью выздоровеет. Об этом она рассказала своей бабушке во время телефонного звонка на родину. Елене Яковлевне Ск